In [ ]:
XH = [3]
ZL = 10
from SY import *
HX.LB(XH, ZL)

# 64

  - 天
    - 天
      - 乾
    - 泽
      - 屯
    - 火
      - 同人
    - 雷
      - 无妄
    - 风
      - 讼
    - 水
      - 需
    - 山
      - 大畜
    - 地
      - 泰

  - 泽
    - 天
      - 夬
    - 泽
      - 兑
    - 火
      - 革
    - 雷
      - 随
    - 风
      - 益
    - 水
      - 困
    - 山
      - 咸
    - 地
      - 萃

  - 火
    - 天
      - 大有
    - 泽
      - 睽
    - 火
      - 离
    - 雷
      - 豫
    - 风
      - 家人
    - 水
      - 既济
    - 山
      - 贲
    - 地
      - 明夷

  - 雷
    - 天
      - 无妄
    - 泽
      - 随
    - 火
      - 豫
    - 雷
      - 震
    - 风
      - 益
    - 水
      - 解
    - 山
      - 小过
    - 地
      - 复

  - 风
    - 天
      - 讼
    - 泽
      - 益
    - 火
      - 家人
    - 雷
      - 益
    - 风
      - 巽
    - 水
      - 涣
    - 山
      - 渐
    - 地
      - 观

  - 水
    - 天
      - 需
    - 泽
      - 困
    - 火
      - 既济
    - 雷
      - 解
    - 风
      - 涣
    - 水
      - 坎
    - 山
      - 蹇
    - 地
      - 比

  - 山
    - 天
      - 大畜
    - 泽
      - 咸
    - 火
      - 贲
    - 雷
      - 小过
    - 风
      - 渐
    - 水
      - 蹇
    - 山
      - 艮
    - 地
      - 谦

  - 地
    - 天
      - 泰
    - 泽
      - 萃
    - 火
      - 明夷
    - 雷
      - 复
    - 风
      - 观
    - 水
      - 比
    - 山
      - 谦
    - 地
      - 坤